**Install Dependencies**

In [ ]:
!pip install -q flask pyngrok transformers sentencepiece accelerate bitsandbytes

**Create app.py**

In [ ]:
%%writefile app.py
# ==============================================
# 🩺 Med Speaking Bot for Healthcare
# Regional Languages + Qwen2.5-7B + NLLB Translation
# Flask Web Deployment
# ==============================================

from flask import Flask, render_template, request
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer as NLLBTokenizer, AutoModelForSeq2SeqLM
from functools import lru_cache
import torch

# ==============================================
# 🌐 Flask App Initialization
# ==============================================
app = Flask(__name__)

# ==============================================
# 🗺️ Language Mapping (Names → NLLB Codes)
# ==============================================
lang_map = {
    "english": "eng_Latn", "en": "eng_Latn",
    "telugu": "tel_Telu", "te": "tel_Telu",
    "hindi": "hin_Deva", "hi": "hin_Deva",
    "tamil": "tam_Taml", "ta": "tam_Taml",
    "kannada": "kan_Knda", "kn": "kan_Knda",
    "malayalam": "mal_Mlym", "ml": "mal_Mlym",
    "marathi": "mar_Deva", "mr": "mar_Deva",
    "bengali": "ben_Beng", "bn": "ben_Beng",
    "gujarati": "guj_Gujr", "gu": "guj_Gujr",
    "punjabi": "pan_Guru", "pa": "pan_Guru",
    "odia": "ory_Orya", "or": "ory_Orya"
}

# ==============================================
# ⚙️ NLLB Translation Model (Lazy-Loaded & Cached)
# ==============================================
@lru_cache(maxsize=1)
def load_nllb():
    """
    Loads the NLLB-200 distilled model and tokenizer once.
    Used for translating between English and regional languages.
    """
    model_name = "facebook/nllb-200-distilled-600M"
    tokenizer = NLLBTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model


# ==============================================
# ⚙️ Qwen2.5-7B-Instruct Model (Lazy-Loaded & Cached)
# ==============================================
@lru_cache(maxsize=1)
def load_qwen():
    """
    Loads the Qwen2.5-7B-Instruct model and tokenizer once.
    Used as the core safety-focused medical assistant.
    """
    model_name = "Qwen/Qwen2.5-7B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        load_in_4bit=True
    )
    return tokenizer, model


# ==============================================
# 🔁 Translation Function (Meaning-Based)
# ==============================================
def translate_nllb(text, src_lang, tgt_lang):
    """
    Uses NLLB to translate text into the target language.
    src_lang is kept in the signature to match original code,
    but NLLB uses only the target language token explicitly.
    """
    nllb_tokenizer, nllb_model = load_nllb()
    inputs = nllb_tokenizer(text, return_tensors="pt")
    output_tokens = nllb_model.generate(
        **inputs,
        forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang)
    )
    return nllb_tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]


# ==============================================
# 🧠 Safe Inference Function (Original Logic Preserved)
# ==============================================
def ask_model(user_message: str, src_lang: str, tgt_lang: str):
    """
    Full pipeline:
    1. Translate user message from source language → English
    2. Ask Qwen2.5-7B with a safety-focused medical prompt
    3. Clean and post-process Qwen output
    4. Translate the answer from English → target language
    """

    # Translate user input → English
    english_input = translate_nllb(user_message, src_lang, "eng_Latn")

    # Original safety-focused system prompt
    prompt = f"""
You are a safety-focused conversational medical assistant.
Follow all the rules below exactly:
- Give simple, general guidance only.
- Do NOT diagnose any condition.
- Do NOT prescribe medicine or mention drug names.
- Do NOT mention medical studies, research papers, or clinical evidence.
- Do NOT add or assume ANY symptoms the user did NOT mention.
- You may briefly reference the user’s last message to maintain a natural conversation.
- Do NOT invent questions or statements the user did NOT say.
- Do NOT add filler questions such as “What should I do”.
- Keep the answer under 3 sentences.
Assistant:
"""

    full_input = prompt + english_input.strip()

    tokenizer, model = load_qwen()
    inputs = tokenizer(full_input, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.2,
        top_p=0.9,
        repetition_penalty=1.25,
        tokenizer=tokenizer
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's answer after "Assistant:"
    if "Assistant:" in response:
        response = response.split("Assistant:", 1)[-1].strip()

    # Remove any echo of the original English input
    if english_input in response:
        response = response.replace(english_input, "").strip()

    # Translate Qwen's answer → target language
    final_output = translate_nllb(response, "eng_Latn", tgt_lang)

    return final_output


# ==============================================
# 🌐 Flask Route — Main Page
# ==============================================
@app.route("/", methods=["GET", "POST"])
def home():
    response = ""
    error = ""
    src_in = ""
    tgt_in = ""
    src_lang_code = ""
    tgt_lang_code = ""

    if request.method == "POST":
        # Get languages and user query from form
        src_in = request.form.get("src_lang", "").strip().lower()
        tgt_in = request.form.get("tgt_lang", "").strip().lower()
        user_query = request.form.get("user_query", "").strip()

        # Map human inputs to NLLB codes (default English if unknown)
        src_lang_code = lang_map.get(src_in, "eng_Latn")
        tgt_lang_code = lang_map.get(tgt_in, "eng_Latn")

        if not user_query:
            error = "⚠️ Please enter a medical question."
        else:
            try:
                response = ask_model(user_query, src_lang_code, tgt_lang_code)
            except Exception as e:
                error = f"⚠️ Error while generating response: {e}"

    return render_template(
        "index.html",
        response=response,
        error=error,
        src_in=src_in,
        tgt_in=tgt_in,
        src_lang_code=src_lang_code,
        tgt_lang_code=tgt_lang_code,
    )


# ==============================================
# 🚀 Run Flask App
# ==============================================
if __name__ == "__main__":
    # Note: In Colab, this will be run behind ngrok.
    app.run(host="0.0.0.0", port=8000, debug=False)

**Create Folders**

In [ ]:
!mkdir -p templates
!mkdir -p static

**HTML Template — templates/index.html**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>🩺 Med Speaking Bot – Regional Languages</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
    <div class="hero-section">
        <div class="overlay"></div>

        <div class="hero-content">
            <h1>🩺 Med Speaking Bot</h1>
            <p>Ask simple health-related questions and get safe, general guidance in regional languages.</p>

            <!-- Language Selection + Query Form -->
            <form method="post" class="card">
                <div class="lang-row">
                    <div class="lang-field">
                        <label for="src_lang">Source language (your input)</label>
                        <input
                            type="text"
                            id="src_lang"
                            name="src_lang"
                            placeholder="e.g., telugu / te"
                            value="{{ src_in }}"
                            required
                        />
                    </div>
                    <div class="lang-field">
                        <label for="tgt_lang">Target language (bot reply)</label>
                        <input
                            type="text"
                            id="tgt_lang"
                            name="tgt_lang"
                            placeholder="e.g., telugu / te"
                            value="{{ tgt_in }}"
                            required
                        />
                    </div>
                </div>

                <textarea
                    name="user_query"
                    rows="4"
                    placeholder="Type your medical query here (e.g., I'm having mild chest discomfort)..."
                    required
                ></textarea>

                {% if error %}
                <p class="error-msg">{{ error }}</p>
                {% endif %}

                <button type="submit" class="btn-primary">Get Response 💬</button>

                {% if src_lang_code or tgt_lang_code %}
                <p class="lang-info">
                    Mapped source code: <strong>{{ src_lang_code }}</strong> |
                    target code: <strong>{{ tgt_lang_code }}</strong>
                </p>
                {% endif %}
            </form>
        </div>
    </div>

    <!-- Response Section -->
    {% if response %}
    <div class="result-section fade-in">
        <div class="result-card">
            <h2>💡 Qwen2.5-7B-Instruct Response</h2>
            <p class="result-text">{{ response }}</p>
        </div>
    </div>
    {% endif %}
</body>
</html>

**CSS — static/style.css**

In [ ]:
%%writefile static/style.css
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

body {
    margin: 0;
    font-family: 'Poppins', sans-serif;
    color: #fff;
}

/* Background hero section */
.hero-section {
    position: relative;
    min-height: 100vh;
    display: flex;
    align-items: center;
    padding: 40px 8%;
    background: linear-gradient(135deg, #1c1f3b, #304c73);
}

.overlay {
    position: absolute;
    inset: 0;
    background: rgba(0,0,0,0.35);
}

/* Main content card */
.hero-content {
    position: relative;
    z-index: 2;
    max-width: 700px;
    background: rgba(255,255,255,0.08);
    border-radius: 24px;
    padding: 28px 32px;
    backdrop-filter: blur(8px);
    box-shadow: 0 10px 30px rgba(0,0,0,0.45);
}

h1 {
    margin: 0 0 10px;
    font-size: 2rem;
    color: #ffdf9e;
}

p {
    margin: 4px 0;
}

/* Form card */
.card {
    margin-top: 18px;
    padding: 16px 18px;
    background: rgba(0,0,0,0.35);
    border-radius: 16px;
    display: flex;
    flex-direction: column;
    gap: 12px;
}

/* Language fields row */
.lang-row {
    display: flex;
    gap: 12px;
    flex-wrap: wrap;
}

.lang-field {
    flex: 1 1 180px;
    display: flex;
    flex-direction: column;
    gap: 6px;
}

.lang-field label {
    font-size: 0.85rem;
}

.lang-field input {
    padding: 8px 10px;
    border-radius: 8px;
    border: none;
    background: rgba(255,255,255,0.95);
    color: #111;
    font-size: 0.9rem;
}

/* Textarea */
textarea {
    width: 100%;
    padding: 10px;
    border-radius: 10px;
    border: none;
    resize: vertical;
    min-height: 100px;
    background: rgba(255,255,255,0.95);
    color: #111;
    font-size: 0.95rem;
}

/* Button */
.btn-primary {
    padding: 10px 12px;
    border: none;
    border-radius: 10px;
    background: linear-gradient(135deg, #00c6ff, #0072ff);
    color: #fff;
    font-weight: 600;
    cursor: pointer;
    font-size: 0.95rem;
    align-self: flex-start;
    transition: 0.25s ease;
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 16px rgba(0,0,0,0.35);
}

/* Info / error */
.error-msg {
    margin-top: 4px;
    color: #ffb3b3;
    font-size: 0.9rem;
}

.lang-info {
    margin-top: 6px;
    font-size: 0.85rem;
    color: #d3e0ff;
}

/* Response section */
.result-section {
    background: #0d111f;
    padding: 40px 20px;
    display: flex;
    justify-content: center;
}

.result-card {
    max-width: 800px;
    width: 100%;
    background: rgba(255,255,255,0.05);
    padding: 24px 28px;
    border-radius: 16px;
    color: #e2e5ff;
}

.result-card h2 {
    margin-top: 0;
}

.result-text {
    white-space: pre-wrap;
    line-height: 1.5;
}

/* Animation */
.fade-in {
    animation: fadeInUp 0.6s ease forwards;
}

@keyframes fadeInUp {
    from { opacity: 0; transform: translateY(18px); }
    to   { opacity: 1; transform: translateY(0); }
}

**Start Flask + ngrok**

In [ ]:
# Kill any running Flask/ngrok processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 617

In [ ]:
# Start Flask in the background
!nohup python app.py > flask.log 2>&1 &

In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok, conf

# Enter your NGROK auth token here
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"  # replace if needed

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)